# FLASK in Google Colab deploying through Heroku
Deploying ML web app on Heroku using Colab.  
Fonte:  
https://medium.com/@amannair723/flask-in-colab-deploying-in-heroku-80a479263b17

In [1]:
# Instala Flask no Colab
!pip install flask-ngrok
%mkdir templates -p
%mkdir static/css -p

In [3]:
# Cria index.html no diretório template
%%writefile templates/index.html
<!DOCTYPE html>
<html >
<!--From https://codepen.io/frytyler/pen/EGdtg-->
<head>
 <meta charset="UTF-8">
 <title>SPAM_API</title>
 <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
</head>
<body>
<div class="login">
    <h1>Spam detection</h1>
    <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
    <textarea id="styled" name="message" rows="6" cols="50" placeholder="Enter Your Message Here" required="required" ></textarea>
    <button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
   </form>
  <br>
  <br>
  {{ prediction_text }}
</div>
</body>
</html>


Overwriting templates/index.html


In [4]:
# Cria arquivo app.py que carrega modelo treinado, recebe dados entrada e faz predição .
%%writefile app.py
#importing libraries
import numpy as np
from flask import Flask, request, jsonify, render_template
from flask_ngrok import run_with_ngrok
import pickle
#creating the flask object
app = Flask(__name__)
run_with_ngrok(app)
#loading the model weights
filename = r"/content/new_nlp_model.pkl"
clf = pickle.load(open(filename, 'rb'))
cv=pickle.load(open(r"/content/new_tranform.pkl",'rb'))
#create routes
@app.route('/')
def home():
   return render_template('index.html')
@app.route('/predict',methods=['POST'])
def predict():
   '''
   For rendering results on HTML GUI
   '''
   if request.method == 'POST':
       message = request.form['message']
       data = [message]
       vect = cv.transform(data).toarray()
       my_prediction = clf.predict(vect)
       if my_prediction == 1:
           output = "a Spam"
       elif my_prediction == 0:
           output = "Not a Spam"
  
   outputs = 'This email is '+output
   return render_template('index2.html', prediction_text=outputs , value=message)
if __name__ == "__main__":
   app.run()


Writing app.py


In [5]:
%%writefile Procfile.txt
web: gunicorn app:app

Writing Procfile.txt


In [6]:
%%writefile requirement.txt
Flask==1.1.1
gunicorn=19.9.0
numpy>=1.9.2
matplotlib>=1.4.3
Pandas>=0.19


Writing requirement.txt
